In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install pytorch-msssim

In [3]:
import glob
import random
import os
import numpy as np
import argparse
import math
import itertools
import sys

import torch
from torch import nn
import torch.nn.functional as F
from torch.utils.data import Dataset
from PIL import Image
import torchvision.transforms as transforms

import torchvision.transforms as transforms
from torchvision.utils import save_image, make_grid

from torch.utils.data import DataLoader
from torch.autograd import Variable

from matplotlib import pyplot as ply

from pytorch_msssim import ssim, ms_ssim, SSIM, MS_SSIM

# Normalization parameters for pre-trained PyTorch models
mean = np.array([0.485, 0.456, 0.406])
std = np.array([0.229, 0.224, 0.225])

In [8]:
class ImageDataset(Dataset):
    def __init__(self, root, shape, transform):
        height, width = shape
        # Transforms for low resolution images and high resolution images
        self.transform = transform

        self.files = sorted(glob.glob(root + "/*.*"))

    def __getitem__(self, index):
        low_path = self.files[index % len(self.files)]
        #high_path = low_path.replace('low','high')
        
        img_low = Image.open(low_path)
        #img_high = Image.open(high_path)
        
        img_low = self.transform(img_low)
        #img_high = self.transform(img_high)
         
        name = low_path[68:]

        return {"low": img_low, "name": name}


    def __len__(self):
        return len(self.files)

In [9]:
root = '/content/drive/MyDrive/ImageSuperResolution/DatasetBatches/test/low'
shape = (400//4,400//4)

transform = transforms.Compose(
            [
                transforms.Resize((shape[0], shape[1]), Image.BICUBIC),
                transforms.ToTensor(),
                #transforms.Normalize(mean, std),
            ]
        )

/usr/local/lib/python3.7/dist-packages/torchvision/transforms/transforms.py:258: UserWarning: Argument interpolation should be of type InterpolationMode instead of int. Please, use InterpolationMode enum.
  "Argument interpolation should be of type InterpolationMode instead of int. "


In [10]:
data = ImageDataset(root, shape, transform)

In [11]:
data[0]

{'low': tensor([[[0.0196, 0.0196, 0.0196,  ..., 0.0196, 0.0078, 0.0039],
          [0.0196, 0.0196, 0.0196,  ..., 0.0196, 0.0078, 0.0039],
          [0.0196, 0.0196, 0.0196,  ..., 0.0196, 0.0078, 0.0039],
          ...,
          [0.0039, 0.0039, 0.0039,  ..., 0.0039, 0.0039, 0.0039],
          [0.0039, 0.0039, 0.0039,  ..., 0.0039, 0.0039, 0.0039],
          [0.0039, 0.0039, 0.0039,  ..., 0.0039, 0.0039, 0.0039]],
 
         [[0.0235, 0.0235, 0.0235,  ..., 0.0275, 0.0078, 0.0039],
          [0.0235, 0.0235, 0.0235,  ..., 0.0275, 0.0078, 0.0039],
          [0.0235, 0.0235, 0.0235,  ..., 0.0275, 0.0118, 0.0039],
          ...,
          [0.0039, 0.0039, 0.0039,  ..., 0.0039, 0.0039, 0.0039],
          [0.0039, 0.0039, 0.0039,  ..., 0.0039, 0.0039, 0.0039],
          [0.0039, 0.0039, 0.0039,  ..., 0.0039, 0.0039, 0.0039]],
 
         [[0.0314, 0.0314, 0.0314,  ..., 0.0235, 0.0078, 0.0039],
          [0.0314, 0.0314, 0.0314,  ..., 0.0235, 0.0078, 0.0039],
          [0.0314, 0.0314, 0.0314

In [13]:
os.makedirs("/content/drive/MyDrive/ImageSuperResolution/DatasetBatches/test/100X100_low", exist_ok=True)

In [14]:
generated_image_root = "/content/drive/MyDrive/ImageSuperResolution/DatasetBatches/test/100X100_low/"

In [ ]:
for i in range(len(data)):
    img = data[i]
    save_image(img['low'], generated_image_root + img['name'])